<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

在前几篇中，我们了解了torch如何通过trace或者script两种方式将nn.Module转换为nn.ScriptModel，并且通过代码示例解释了两者之间的区别。此外，还简单说明了onnx算子和如何使用torch.onnx.export将nn.Module转换为onnx算子，并且通过onnxruntime运行模型。在这一篇我们将补全ScriptModel到onnx算子的线，即学习ScriptModel中的aten算子如何映射到onnx算子上。